# Digital signature in Bitcoin

In [1]:
import bitcoin
import ecdsa

In [2]:
#secp256k1 elliptic curve
_p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
_n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
_b = 0x0000000000000000000000000000000000000000000000000000000000000007
_a = 0x0000000000000000000000000000000000000000000000000000000000000000
_Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
_Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8

curve = ecdsa.ellipticcurve.CurveFp(_p, _a, _b)
generator = ecdsa.ellipticcurve.Point(curve, _Gx, _Gy, _n)
oid = (1, 3, 132, 0, 10)
SECP256k1 = ecdsa.curves.Curve("SECP256k1", curve, generator, oid)

In [3]:
#Initializing necessary data
priv_key = 'b793394fba80ae48cea022e816a0681d68ab34f5fab8e996d7e455f8dade48bc'
pub_key_point = int(priv_key,16) * generator
tx = 'Transaction data'
print('Private key:', priv_key, sep='\n', end='\n\n')
print('Public key point:', pub_key_point, sep='\n', end='\n\n')
print('Example transction data:', "'" + tx + "'", sep='\n')

Private key:
b793394fba80ae48cea022e816a0681d68ab34f5fab8e996d7e455f8dade48bc

Public key point:
(81852358409583078132913769522892336249086096395542025604694199336325523479920,80342473401992836716531609393971973732548147875775607464919304601980854872630)

Example transction data:
'Transaction data'


In [4]:
#Computing transaction hash with priorly added SIGHASH type
tx_hash = bitcoin.txhash(tx, hashcode=bitcoin.SIGHASH_ALL)
print('Transaction hash:', tx_hash)

Transaction hash: 7cc03f2b1f466374e8691e43f69ceff4680d5760a1fcb22e387c4f9798e4b482


In [5]:
#Generating random number k (ephemeral private key)
k = bitcoin.deterministic_generate_k(bitcoin.txhash(tx), priv_key)
print('k =', k)

k = 10367737156050198742371273494331617727131814076475671317911472787261859672112


In [6]:
#Computing point P (ephemeral public key)
P = k * generator
R = P.x() % _n
print('P =', P)
print('R =', R)

P = (7958611498930676801754267071725705291801676223419638510268672625381072649783,1922522205416409897854082188621196197494542542592200168483806142459855183244)
R = 7958611498930676801754267071725705291801676223419638510268672625381072649783


$$P = k \times G$$
$$R = P_x \mod{n}$$
$$ n \times G = O$$ 

In [7]:
#Converting transaction hash and private key to integers
tx_hash = int(tx_hash, 16)
priv_key = int(priv_key, 16)

In [8]:
#Computing S using modular multiplicative inverse of k
k_inv = bitcoin.inv(k, _n)
S = (k_inv * (tx_hash + priv_key*R)) % _n
print('S =', S)

S = 1334613320548274216532725684432922860199418151669975572704907084091578940890


$$ S = k^{-1}(hash(tx) + privkey*R) \mod{n}$$

In [9]:
#Digital signature
print('Digital signature(R,S):', (R,S))

Digital signature(R,S): (7958611498930676801754267071725705291801676223419638510268672625381072649783, 1334613320548274216532725684432922860199418151669975572704907084091578940890)


In [10]:
#Digital signature verification using public key and transaction hash
inv_S = bitcoin.inv(S, _n)
verification_P = (inv_S*tx_hash)*generator + (inv_S*R)*pub_key_point
print('verification_P.x =', verification_P.x(), end='\n\n')
if verification_P.x() == R:
    print('Succesfully verificated')

verification_P.x = 7958611498930676801754267071725705291801676223419638510268672625381072649783

Succesfully verificated


$$ P^{verification} = S^{-1}hash(tx) \times G \space + S^{-1}R \times pubkey $$
$$ \text{Verificated if } P^{verification}_x = R$$